In [46]:
import time
# import IPython
import os
import json
from datetime import datetime
import pyswarm
import pandas

import joblib
import numpy as np
import matplotlib.pyplot as plt
# from mpl_toolkits.mplot3d import Axes3D
import tensorflow.compat.v2 as tf
import tensorflow_probability as tfp
from tensorflow import keras
import tensorflow.keras.backend as K


from keras.models import Sequential, Model
# from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization, Lambda, Layer
# from tensorflow.keras import losses
# from keras.utils.vis_utils import plot_model, model_to_dot

from sklearn.preprocessing import StandardScaler, MinMaxScaler
import sklearn.preprocessing

# from sklearn.model_selection import train_test_split
# from sklearn.metrics import r2_score, mean_absolute_error, median_absolute_error

K.set_floatx('float32')

tfb = tfp.bijectors
tfd = tfp.distributions
tfk = tfp.math.psd_kernels

tf.enable_v2_behavior()

from tester.tester import Tester

In [4]:
log_path = os.path.join('.','2021_12_18_17_02_54')

In [5]:
path = os.path.join(log_path, 'run_stats.txt')
with open(path, 'r') as file:
    run_stats = json.load(file)    

    
num_samples = run_stats['num_samples']
training_proportion = run_stats['training_proportion']
optimization_proportion = 1. - training_proportion

log_std_coefficient = run_stats['log_std_coefficient']
constant_std_coefficient = run_stats['constant_std_coefficient']

run_stats

{'num_samples': 90,
 'training_proportion': 0.4,
 'log_std_coefficient': 4.0,
 'constant_std_coefficient': 20.0}

In [6]:
path = os.path.join(log_path, 'training.npyz')
with open(path, 'rb') as file:
    training_data = np.load(file)
    training_Xs=training_data['points']
    training_ys=training_data['values']
    norm_training_Xs=training_data['norm_points']
    norm_training_ys=training_data['norm_values']
    lls_=training_data['loss']
    amplitude= training_data['amplitude']
    relevances= training_data['relevances']
    noise_var= training_data['noise']
    

In [20]:
path=os.path.join(log_path,'scaler_X.pkl')
transform_X = joblib.load(path)
path=os.path.join(log_path,'scaler_y.pkl')
transform_y = joblib.load(path)


/home/llarrauriborroto/Documents/Projects/advanced-bayesian-opt/lib/python3.8/site-packages/sklearn/base.py:313: UserWarning: Trying to unpickle estimator StandardScaler from version 0.21.2 when using version 0.22. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


In [13]:
norm_training_Xs


array([[ 0.47204655, -0.424117  , -0.4290627 ],
       [ 1.4353002 , -0.7703848 ,  1.1779659 ],
       [ 1.2311457 ,  1.1854705 , -1.345039  ],
       [ 0.02292409,  1.4072273 ,  0.96867   ],
       [ 0.39290458,  0.07622249,  0.53100765],
       [ 1.1342335 ,  0.20291275,  1.55884   ],
       [-0.5572644 , -1.9319245 ,  0.36695644],
       [-1.4754751 ,  0.30100176,  1.4472328 ],
       [-0.9571947 ,  0.4965771 , -0.67384315],
       [-0.99550146,  1.5792048 ,  1.4990652 ],
       [ 0.695943  , -1.4284133 ,  0.5185804 ],
       [-0.06327949, -0.16393444,  1.0606676 ],
       [ 0.19117436, -1.758254  , -0.39365   ],
       [-1.4166954 ,  0.5353121 , -0.13566571],
       [ 0.8335138 , -0.70120764, -1.0123981 ],
       [-1.4259027 ,  0.45895618,  0.10292322],
       [-0.2505929 , -0.15489891, -0.6159126 ],
       [-1.4538466 , -0.81437695, -1.3159847 ],
       [ 1.5933108 , -0.56150264, -1.0885805 ],
       [-0.15175854,  1.0218003 , -0.4219295 ],
       [ 1.2697424 ,  0.39618462,  0.599

In [15]:
kernel = tfk.ExponentiatedQuadratic(amplitude, length_scale=1.0)
kernel = tfk.FeatureScaled(kernel, scale_diag=relevances)
gp_model = tfd.GaussianProcessRegressionModel(
    kernel=kernel,
    index_points=[[1.2353002 , -0.7703848 ,  1.1779659]],
    observation_index_points=norm_training_Xs,
    observations=norm_training_ys,
    observation_noise_variance=noise_var,
    predictive_noise_variance=0.,
    jitter=1e-4)

gp_model.mean()

<tf.Tensor: shape=(), dtype=float32, numpy=1.0213013>

In [18]:
path = os.path.join(log_path, 'optimization_rounds.npyz')
with open(path, 'rb') as file:
    optimization_data = np.load(file)
    real_values=optimization_data['values']
    real_choices=optimization_data['choices']
    mean_regrets=optimization_data['mean_regrets']


In [36]:
values= transform_y.transform(real_values.reshape(-1,1))
choices= transform_X.transform(real_choices)
values=values.flatten()


In [62]:
with open('./validation_samples.csv', newline='') as csvfile:
    reader = pandas.read_csv(csvfile,delimiter=';')

    

In [74]:
nom_cols =reader['corner']=='nom'
validation_frame= reader[nom_cols]
validation_frame


,Parameter Label,Pattern,Run,Measurement Name,corner,p_vdda_evr,p_vdda_hpbg,p_vddpd,p_vref,p_vss,temperature,trimbg,trimcur,Unnamed: 13,Value,MC Mean,MC Stddev,Cp,Cpk,Unit
0,I_ref_20n_untrimmed,hpbg_startup_trimall2,RUN0001,I_ref_20n_untrimmed,nom,2.2,2.2,1.08,0.6,0,-40,64,16,NaN,-0.000019,,,,,A
1,I_ref_20n_untrimmed,hpbg_startup_trimall2,RUN0002,I_ref_20n_untrimmed,nom,2.2,2.2,1.12,0.6,0,-40,64,16,NaN,-0.000019,,,,,A
2,I_ref_20n_untrimmed,hpbg_startup_trimall2,RUN0003,I_ref_20n_untrimmed,nom,2.2,2.2,1.16,0.6,0,-40,64,16,NaN,-0.000019,,,,,A
3,I_ref_20n_untrimmed,hpbg_startup_trimall2,RUN0004,I_ref_20n_untrimmed,nom,2.2,2.2,1.20,0.6,0,-40,64,16,NaN,-0.000019,,,,,A
4,I_ref_20n_untrimmed,hpbg_startup_trimall2,RUN0005,I_ref_20n_untrimmed,nom,2.2,2.2,1.24,0.6,0,-40,64,16,NaN,-0.000019,,,,,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5827,V_ref_1v133_untrimmed,hpbg_startup_trimall2,RUN0968,V_ref_1v133_untrimmed,nom,2.8,2.8,1.24,0.6,0,175,64,16,NaN,1.135183,,,,,V
5828,V_ref_1v133_untrimmed,hpbg_startup_trimall2,RUN0969,V_ref_1v133_untrimmed,nom,2.8,2.8,1.28,0.6,0,175,64,16,NaN,1.135178,,,,,V
5829,V_ref_1v133_untrimmed,hpbg_startup_trimall2,RUN0970,V_ref_1v133_untrimmed,nom,2.8,2.8,1.32,0.6,0,175,64,16,NaN,1.135178,,,,,V
5830,V_ref_1v133_untrimmed,hpbg_startup_trimall2,RUN0971,V_ref_1v133_untrimmed,nom,2.8,2.8,1.36,0.6,0,175,64,16,NaN,1.135173,,,,,V


In [82]:
for index, row in validation_frame.iterrows():
    print(row['Value'],row['p_vddpd'], row['p_vddpd'])

# validation_frame.loc[3].at['p_']


-1.93999248e-05 1.08
-1.93999351e-05 1.12
-1.93999233e-05 1.16
-1.93999378e-05 1.2
-1.93999235e-05 1.24
-1.9399925e-05 1.28
-1.93999248e-05 1.32
-1.93999243e-05 1.36
-1.93999245e-05 1.41
-1.94128258e-05 1.08
-1.94128316e-05 1.12
-1.94128259e-05 1.16
-1.94128262e-05 1.2
-1.94128267e-05 1.24
-1.94128259e-05 1.28
-1.94128247e-05 1.32
-1.94128393e-05 1.36
-1.94128259e-05 1.41
-1.94172213e-05 1.08
-1.94172362e-05 1.12
-1.94172155e-05 1.16
-1.94172255e-05 1.2
-1.94172221e-05 1.24
-1.94172235e-05 1.28
-1.94172052e-05 1.32
-1.94172042e-05 1.36
-1.94172119e-05 1.41
-1.94098064e-05 1.08
-1.9409788e-05 1.12
-1.94097994e-05 1.16
-1.94097897e-05 1.2
-1.94097995e-05 1.24
-1.94098314e-05 1.28
-1.94098334e-05 1.32
-1.94098334e-05 1.36
-1.94098062e-05 1.41
-1.94008137e-05 1.08
-1.9400809e-05 1.12
-1.94008211e-05 1.16
-1.94008035e-05 1.2
-1.94008035e-05 1.24
-1.94008001e-05 1.28
-1.94008017e-05 1.32
-1.94008078e-05 1.36
-1.94008429e-05 1.41
-1.93801566e-05 1.08
-1.93801484e-05 1.12
-1.93801787e-05 1.16


-1.9354649e-05 1.28
-1.9354593e-05 1.32
-1.93546455e-05 1.36
-1.93545781e-05 1.41
-1.93178448e-05 1.08
-1.93178757e-05 1.12
-1.93178503e-05 1.16
-1.93178744e-05 1.2
-1.93179029e-05 1.24
-1.93178756e-05 1.28
-1.93178761e-05 1.32
-1.93179037e-05 1.36
-1.93178909e-05 1.41
-1.9273354e-05 1.08
-1.927338e-05 1.12
-1.92733234e-05 1.16
-1.92733453e-05 1.2
-1.92733632e-05 1.24
-1.92733626e-05 1.28
-1.92733804e-05 1.32
-1.92733806e-05 1.36
-1.9273301e-05 1.41
-1.92212581e-05 1.08
-1.92212707e-05 1.12
-1.92212645e-05 1.16
-1.92212588e-05 1.2
-1.92212649e-05 1.24
-1.92212702e-05 1.28
-1.92212697e-05 1.32
-1.92212716e-05 1.36
-1.92212629e-05 1.41
-1.91620657e-05 1.08
-1.91620823e-05 1.12
-1.91620443e-05 1.16
-1.91620916e-05 1.2
-1.91620713e-05 1.24
-1.91620667e-05 1.28
-1.9162066e-05 1.32
-1.91620723e-05 1.36
-1.9162066e-05 1.41
-1.91133285e-05 1.08
-1.911333e-05 1.12
-1.9113332e-05 1.16
-1.91133324e-05 1.2
-1.91133292e-05 1.24
-1.91133318e-05 1.28
-1.91133318e-05 1.32
-1.91133326e-05 1.36
-1.91133

1.13641572 1.28
1.13640633 1.32
1.13641827 1.36
1.13640815 1.41
1.13644924 1.08
1.13646225 1.12
1.13647707 1.16
1.13644763 1.2
1.13645288 1.24
1.13646913 1.28
1.13645099 1.32
1.13646341 1.36
1.136453 1.41
1.13650246 1.08
1.13649363 1.12
1.13650549 1.16
1.13647593 1.2
1.13646382 1.24
1.13648578 1.28
1.13646114 1.32
1.13645684 1.36
1.13645771 1.41
1.1365154 1.08
1.1364443 1.12
1.13648775 1.16
1.13650307 1.2
1.13646152 1.24
1.13647376 1.28
1.13646869 1.32
1.13643638 1.36
1.13643448 1.41
1.13644619 1.08
1.13639747 1.12
1.13642228 1.16
1.13644915 1.2
1.13643043 1.24
1.13638733 1.28
1.13640374 1.32
1.1363745 1.36
1.13636915 1.41
1.13630068 1.08
1.13631256 1.12
1.13627772 1.16
1.13632357 1.2
1.13633301 1.24
1.13628125 1.28
1.13627688 1.32
1.13627899 1.36
1.13626012 1.41
1.13614334 1.08
1.13614216 1.12
1.13605557 1.16
1.13611269 1.2
1.13613211 1.24
1.13612795 1.28
1.13609195 1.32
1.1361058 1.36
1.13609237 1.41
1.13583962 1.08
1.13583234 1.12
1.13578027 1.16
1.13579532 1.2
1.13583704 1.24
1.135